In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import numpy as np
from tqdm import tqdm
import pandas as pd
import math


In [8]:
path = 'chromedriver.exe'
options = webdriver.ChromeOptions()

In [ ]:
chrome_service = Service(executable_path=path)
chrome = webdriver.Chrome(options = options, service = chrome_service)

In [ ]:
chrome.get('https://www.nasmm.org/find-a-move-manager/')

In [175]:
x=0
for i in range(1,1000):
    
    if len(chrome.find_elements(By.XPATH,'/html/body/div[3]/div/section/div/div[3]/div/div[1]/div[2]/div/ul/li[{}]'.format(i)))>0:
        x+=1
    else:
        break

In [176]:
x

51

In [177]:
links=[]
for m in tqdm(range(1,x+1)):
    ele = chrome.find_elements(By.XPATH,'/html/body/div[3]/div/section/div/div[3]/div/div[1]/div[2]/div/ul/li[{}]/a'.format(m))[0]
    links.append(ele.get_attribute('href'))
        

100%|██████████| 51/51 [00:01<00:00, 25.57it/s]


In [178]:
links

['https://www.nasmm.org/find-a-move-manager/list.cfm?state=AL&country=US',
 'https://www.nasmm.org/find-a-move-manager/list.cfm?state=AK&country=US',
 'https://www.nasmm.org/find-a-move-manager/list.cfm?state=AZ&country=US',
 'https://www.nasmm.org/find-a-move-manager/list.cfm?state=AR&country=US',
 'https://www.nasmm.org/find-a-move-manager/list.cfm?state=CA&country=US',
 'https://www.nasmm.org/find-a-move-manager/list.cfm?state=CO&country=US',
 'https://www.nasmm.org/find-a-move-manager/list.cfm?state=CT&country=US',
 'https://www.nasmm.org/find-a-move-manager/list.cfm?state=DE&country=US',
 'https://www.nasmm.org/find-a-move-manager/list.cfm?state=DC&country=US',
 'https://www.nasmm.org/find-a-move-manager/list.cfm?state=FL&country=US',
 'https://www.nasmm.org/find-a-move-manager/list.cfm?state=GA&country=US',
 'https://www.nasmm.org/find-a-move-manager/list.cfm?state=HI&country=US',
 'https://www.nasmm.org/find-a-move-manager/list.cfm?state=ID&country=US',
 'https://www.nasmm.org/f

In [180]:
CN = []
N = []
A = []
PN = []
APN = []
EM = []

for link in links:

    chrome.get(link)
    try:
        x = chrome.find_element(By.XPATH,'/html/body/div[3]/div/section/div/div[2]/div/div[1]').text
    except:
        continue
    x1 = int(x.split()[1].split('-')[0])
    x2 = int(x.split()[1].split('-')[1])
    x3 = int(x.split()[3])
    num_pages = math.ceil(x3/(x2-x1+1))

    for k in range(1,num_pages+1):
        x = chrome.find_element(By.XPATH,'/html/body/div[3]/div/section/div/div[2]/div/div[1]').text
        x1 = int(x.split()[1].split('-')[0])
        x2 = int(x.split()[1].split('-')[1])
        x3 = int(x.split()[3])
        num_people = (x2-x1+1)
        
        for j in tqdm(range(1,num_people+1)):
            xpath1 = '/html/body/div[3]/div/section/div/div[2]/div/ul/li[{}]'.format(j+(k-1)*10)
            company_name = chrome.find_element(By.XPATH, xpath1).text.split('\n')[0]
            name = chrome.find_element(By.XPATH, xpath1).text.split('\n')[1]
            address = chrome.find_element(By.XPATH, xpath1).text.split('\n')[2]
            phone = chrome.find_element(By.XPATH, xpath1).text.split('\n')[4]
            info = chrome.find_element(By.XPATH, xpath1).text.split('\n')
            try:
                alternate_phone = [i for i in info if 'Alternative Phone:' in i][0]
            except:
                alternate_phone = np.nan
            try:
                email = [i for i in info if 'Email:' in i][0]
            except:
                email = np.nan
            CN.append(company_name)
            N.append(name)
            A.append(address)
            PN.append(phone)
            APN.append(alternate_phone)
            EM.append(email)
        chrome.find_element(By.CLASS_NAME,'next_link').click()

100%|██████████| 1/1 [00:00<00:00,  7.05it/s]


In [181]:
df = pd.DataFrame(CN, columns = ['Company Name'])
df['Name'] = N
df['Email'] = EM
df['Phone'] = PN
df['Alternative Phone'] = APN
df['Address'] = A

In [182]:
df

,Company Name,Name,Email,Phone,Alternative Phone,Address
0,"Aging in Place Services, LLC",Angie Durr,Email: info@aginginplaceservices.net,Phone: 629-999 2477,NaN,"Nashville, TN 37205"
1,Caring Transitions of Huntsville,Robert Alley,Email: ralley@caringtransitions.com,Phone: 256-609-0904,NaN,"Huntsville, AL 35802"
2,Life Simplified,Karen Wright,Email: karen@lifesimplifiedllc.com,Phone: 256-316-1655,NaN,"Huntsville, AL 35815"
3,Silver Service LLC,"Katrina Ranieri, SMM-C",Email: emily@silverservicega.com,Phone: 706-575-6648,Alternative Phone: 706-570-0503,"Columbus, GA 31909"
4,"Smooth Mooove Senior Relocation Services, Inc.",Adrienne Peppler,Email: adrienne@wemoveseniors.com,Phone: 6784766666,NaN,"Stone Mountain, GA 30087"
...,...,...,...,...,...,...
662,Revere Relocation Solutions LLC,Sunni Dalgord,Email: sunni@revererelocation.com,Phone: 920-664-0784,NaN,"Suamico, WI 54313"
663,Rose's Daughters Inc.,Tommy Misener,Email: tommy@rosesdaughters.com,,Alternative Phone: 651-373-1323,Edmond McGill
664,Segues LLC Madison,Dawn Eyres,Email: transitions@segues.net,Phone: 608-345-1942,"Alternative Phone: 262-442-4200 - Milwaukee, WI","Sun Prairie, WI 53590"
665,"Segues, LLC",Dawn Eyre,Email: dawn@segues.net,Phone: 262-442-4200,Alternative Phone: 608-345-1942 Madison Office,"Germantown, WI 53022"
